# 🔍 Data Exploration & Discovery: Uncovering Natural Stories

**Objective**: Let the data speak first. Before crafting narratives, we'll explore our master datasets to discover what stories emerge naturally from patterns, distributions, and relationships.

**Approach**: Data-driven storytelling - starting with comprehensive visualization and pattern discovery to build authentic narratives grounded in genuine data insights.

---

**Master Datasets**:
- `risk_assessment_complete.csv` - Sub-regional risk analysis (4,147 regions)
- `country_summary.csv` - Country-level aggregations  
- Atlas Explorer data fusion results

Let's discover what stories our data wants to tell us!

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# Configure visualization settings
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("📚 Libraries loaded successfully!")
print("🎨 Visualization settings configured")
print("🔍 Ready for data exploration!")

: 

In [ ]:
# Load and Examine Master Data
print("📂 LOADING MASTER DATASETS")
print("=" * 50)

# Load the primary datasets
try:
    # Main risk assessment data
    risk_data = pd.read_csv('../data/processed/risk_assessment_complete.csv')
    print(f"✅ Risk assessment data loaded: {risk_data.shape}")
    
    # Country-level summary
    country_data = pd.read_csv('../submission/data/country_summary.csv')
    print(f"✅ Country summary data loaded: {country_data.shape}")
    
    # Try to load other available datasets
    try:
        compound_risk = pd.read_csv('../data/processed/compound_risk_assessment.csv')
        print(f"✅ Compound risk data loaded: {compound_risk.shape}")
    except FileNotFoundError:
        print("ℹ️ Compound risk data not found")
        compound_risk = None
        
    try:
        master_atlas = pd.read_csv('../data/processed/master_atlas_data_cleaned.csv')
        print(f"✅ Master atlas data loaded: {master_atlas.shape}")
    except FileNotFoundError:
        print("ℹ️ Master atlas data not found")
        master_atlas = None
    
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")
    print("Please ensure the data files are in the correct location")

print("\n🔍 INITIAL DATA EXAMINATION")
print("=" * 50)

# Examine the main risk data structure
print("\n📊 RISK DATA OVERVIEW:")
print(f"Shape: {risk_data.shape}")
print(f"Columns: {len(risk_data.columns)}")
print("\nColumn names:")
for i, col in enumerate(risk_data.columns, 1):
    print(f"  {i:2d}. {col}")

# Basic info about data types
print(f"\n📈 DATA TYPES:")
print(risk_data.dtypes.value_counts())

print(f"\n🌍 GEOGRAPHIC COVERAGE:")
if 'country' in risk_data.columns:
    print(f"Countries: {risk_data['country'].nunique()}")
    print(f"Sample countries: {sorted(risk_data['country'].unique())[:5]}...")
if 'region' in risk_data.columns:
    print(f"Regions: {risk_data['region'].nunique()}")
if 'sub_region' in risk_data.columns:
    print(f"Sub-regions: {risk_data['sub_region'].nunique()}")

print("\n✨ Data loaded and ready for exploration!")

In [ ]:
# Data Preprocessing and Cleaning
print("🧹 DATA PREPROCESSING & QUALITY CHECK")
print("=" * 50)

# Check for missing values
print("\n⚠️ MISSING VALUES ANALYSIS:")
missing_data = risk_data.isnull().sum()
missing_percentage = (missing_data / len(risk_data)) * 100

# Create missing data summary
missing_summary = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_percentage
}).sort_values('Missing_Percentage', ascending=False)

# Show only columns with missing data
missing_cols = missing_summary[missing_summary['Missing_Count'] > 0]
if len(missing_cols) > 0:
    print("Columns with missing values:")
    for col, row in missing_cols.head(10).iterrows():
        print(f"  {col}: {row['Missing_Count']:,} ({row['Missing_Percentage']:.1f}%)")
else:
    print("✅ No missing values found!")

# Identify numeric and categorical columns
numeric_cols = risk_data.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = risk_data.select_dtypes(include=['object']).columns.tolist()

print(f"\n📊 COLUMN TYPES:")
print(f"Numeric columns: {len(numeric_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

# Look for potential key risk indicators
risk_keywords = ['risk', 'score', 'vulnerability', 'hazard', 'exposure']
potential_risk_cols = [col for col in risk_data.columns 
                      if any(keyword in col.lower() for keyword in risk_keywords)]

print(f"\n🎯 POTENTIAL RISK INDICATORS ({len(potential_risk_cols)} found):")
for col in potential_risk_cols:
    if col in numeric_cols:
        values = risk_data[col].dropna()
        print(f"  {col}: {values.min():.3f} - {values.max():.3f} (mean: {values.mean():.3f})")

# Identify geographic columns
geo_keywords = ['country', 'region', 'admin', 'lat', 'lon', 'coord']
geo_cols = [col for col in risk_data.columns 
           if any(keyword in col.lower() for keyword in geo_keywords)]

print(f"\n🌍 GEOGRAPHIC COLUMNS ({len(geo_cols)} found):")
for col in geo_cols:
    print(f"  {col}: {risk_data[col].dtype}")

print("\n✅ Data preprocessing complete!")

In [ ]:
# Basic Statistical Summary
print("📈 DESCRIPTIVE STATISTICS ANALYSIS")
print("=" * 50)

# Generate comprehensive statistics for numeric columns
numeric_stats = risk_data[numeric_cols].describe()
print("\n📊 NUMERIC VARIABLES SUMMARY:")
print(numeric_stats.round(3))

# Focus on key risk indicators if they exist
if potential_risk_cols:
    print(f"\n🎯 KEY RISK INDICATORS DETAILED STATS:")
    risk_numeric = [col for col in potential_risk_cols if col in numeric_cols]
    
    for col in risk_numeric[:5]:  # Show top 5 risk columns
        values = risk_data[col].dropna()
        if len(values) > 0:
            print(f"\n{col}:")
            print(f"  Count: {len(values):,}")
            print(f"  Mean: {values.mean():.4f}")
            print(f"  Std: {values.std():.4f}")
            print(f"  Min: {values.min():.4f}")
            print(f"  25%: {values.quantile(0.25):.4f}")
            print(f"  50%: {values.quantile(0.50):.4f}")
            print(f"  75%: {values.quantile(0.75):.4f}")
            print(f"  Max: {values.max():.4f}")

# Categorical variables summary
print(f"\n📋 CATEGORICAL VARIABLES SUMMARY:")
for col in categorical_cols[:5]:  # Show top 5 categorical columns
    unique_count = risk_data[col].nunique()
    most_common = risk_data[col].value_counts().head(3)
    print(f"\n{col}:")
    print(f"  Unique values: {unique_count}")
    print(f"  Most common:")
    for value, count in most_common.items():
        percentage = (count / len(risk_data)) * 100
        print(f"    {value}: {count:,} ({percentage:.1f}%)")

# Check for potential data quality issues
print(f"\n🔍 DATA QUALITY CHECKS:")

# Check for duplicate rows
duplicates = risk_data.duplicated().sum()
print(f"Duplicate rows: {duplicates:,}")

# Check for negative values in risk scores (if they should be positive)
for col in risk_numeric:
    if 'score' in col.lower() or 'risk' in col.lower():
        negative_count = (risk_data[col] < 0).sum()
        if negative_count > 0:
            print(f"Negative values in {col}: {negative_count:,}")

# Check for extremely high outliers
for col in risk_numeric:
    if col in numeric_cols:
        values = risk_data[col].dropna()
        if len(values) > 0:
            q99 = values.quantile(0.99)
            outliers = (values > q99 * 3).sum()  # Values > 3x the 99th percentile
            if outliers > 0:
                print(f"Extreme outliers in {col}: {outliers:,}")

print("\n✅ Statistical analysis complete!")

In [ ]:
# Univariate Visualizations
print("📊 UNIVARIATE ANALYSIS: Understanding Individual Variables")
print("=" * 60)

# Create comprehensive univariate visualizations
def create_distribution_plots():
    """Create distribution plots for key numeric variables"""
    
    # Select top risk indicators for detailed analysis
    key_risk_cols = [col for col in potential_risk_cols if col in numeric_cols][:4]
    
    if len(key_risk_cols) == 0:
        print("⚠️ No numeric risk columns found, using first 4 numeric columns")
        key_risk_cols = numeric_cols[:4]
    
    # Create subplots for distributions
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[col.replace('_', ' ').title() for col in key_risk_cols],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    for i, col in enumerate(key_risk_cols):
        row = (i // 2) + 1
        col_pos = (i % 2) + 1
        
        # Get clean data
        data = risk_data[col].dropna()
        
        # Create histogram
        fig.add_trace(
            go.Histogram(
                x=data,
                name=col,
                marker_color=colors[i],
                opacity=0.7,
                nbinsx=30
            ),
            row=row, col=col_pos
        )
    
    fig.update_layout(
        title="Distribution of Key Risk Variables",
        height=600,
        showlegend=False
    )
    
    return fig

# Create and display distribution plots
dist_fig = create_distribution_plots()
dist_fig.show()

# Box plots for identifying outliers
def create_box_plots():
    """Create box plots to identify outliers and distribution shape"""
    
    key_cols = [col for col in potential_risk_cols if col in numeric_cols][:6]
    if len(key_cols) == 0:
        key_cols = numeric_cols[:6]
    
    fig = go.Figure()
    
    for i, col in enumerate(key_cols):
        data = risk_data[col].dropna()
        fig.add_trace(go.Box(
            y=data,
            name=col.replace('_', ' ').title(),
            boxpoints='outliers'
        ))
    
    fig.update_layout(
        title="Box Plots: Distribution Shape and Outliers",
        yaxis_title="Values",
        height=500
    )
    
    return fig

box_fig = create_box_plots()
box_fig.show()

# Geographic distribution if country data exists
if 'country' in risk_data.columns:
    print(f"\n🌍 GEOGRAPHIC DISTRIBUTION:")
    
    country_counts = risk_data['country'].value_counts().head(15)
    
    fig = px.bar(
        x=country_counts.values,
        y=country_counts.index,
        orientation='h',
        title="Number of Sub-regions by Country (Top 15)",
        labels={'x': 'Number of Sub-regions', 'y': 'Country'}
    )
    fig.update_layout(height=500)
    fig.show()
    
    print(f"Countries with most sub-regions:")
    for country, count in country_counts.head(5).items():
        percentage = (count / len(risk_data)) * 100
        print(f"  {country}: {count:,} ({percentage:.1f}%)")

print("\n✅ Univariate analysis complete!")

In [ ]:
# Bivariate Analysis and Correlations
print("🔗 BIVARIATE ANALYSIS: Discovering Relationships")
print("=" * 50)

# Calculate correlation matrix for numeric variables
numeric_data = risk_data[numeric_cols].select_dtypes(include=[np.number])

if len(numeric_data.columns) > 1:
    # Correlation matrix
    correlation_matrix = numeric_data.corr()
    
    # Create interactive correlation heatmap
    fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.columns,
        colorscale='RdBu',
        zmid=0,
        text=correlation_matrix.round(3).values,
        texttemplate='%{text}',
        textfont={"size": 10},
        hoverongaps=False
    ))
    
    fig.update_layout(
        title="Correlation Matrix: Numeric Variables",
        width=800,
        height=600
    )
    
    fig.show()
    
    # Find strongest correlations
    print("\n🔥 STRONGEST CORRELATIONS:")
    
    # Get upper triangle of correlation matrix
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)
    correlations = correlation_matrix.where(mask).stack().sort_values(key=abs, ascending=False)
    
    print("Top 10 strongest correlations:")
    for i, (pair, corr) in enumerate(correlations.head(10).items()):
        var1, var2 = pair
        print(f"  {i+1:2d}. {var1} ↔ {var2}: {corr:.3f}")

# Risk factors relationship analysis
if potential_risk_cols:
    print(f"\n🎯 RISK FACTORS RELATIONSHIPS:")
    
    # Select key risk variables for detailed analysis
    key_risk_vars = [col for col in potential_risk_cols if col in numeric_cols][:4]
    
    if len(key_risk_vars) >= 2:
        # Create scatter plot matrix for risk variables
        fig = make_subplots(
            rows=len(key_risk_vars), cols=len(key_risk_vars),
            subplot_titles=[col.replace('_', ' ').title() for col in key_risk_vars]
        )
        
        for i, var1 in enumerate(key_risk_vars):
            for j, var2 in enumerate(key_risk_vars):
                if i != j:
                    # Scatter plot
                    clean_data = risk_data[[var1, var2]].dropna()
                    
                    fig.add_trace(
                        go.Scatter(
                            x=clean_data[var2],
                            y=clean_data[var1],
                            mode='markers',
                            marker=dict(size=3, opacity=0.6),
                            name=f"{var1} vs {var2}",
                            showlegend=False
                        ),
                        row=i+1, col=j+1
                    )
                elif i == j:
                    # Diagonal: histogram
                    data = risk_data[var1].dropna()
                    fig.add_trace(
                        go.Histogram(
                            x=data,
                            name=var1,
                            showlegend=False,
                            opacity=0.7
                        ),
                        row=i+1, col=j+1
                    )
        
        fig.update_layout(
            title="Risk Variables Relationship Matrix",
            height=600,
            showlegend=False
        )
        
        fig.show()

# Geographic patterns analysis
if 'country' in risk_data.columns and potential_risk_cols:
    print(f"\n🌍 GEOGRAPHIC PATTERNS:")
    
    # Select a key risk variable for geographic analysis
    main_risk_var = potential_risk_cols[0] if potential_risk_cols[0] in numeric_cols else numeric_cols[0]
    
    # Country-level risk distribution
    country_risk = risk_data.groupby('country')[main_risk_var].agg(['mean', 'std', 'count']).round(3)
    country_risk = country_risk.sort_values('mean', ascending=False)
    
    print(f"\nTop 10 countries by average {main_risk_var}:")
    for country, row in country_risk.head(10).iterrows():
        print(f"  {country}: {row['mean']:.3f} (±{row['std']:.3f}, n={row['count']})")
    
    # Create geographic risk visualization
    fig = px.bar(
        x=country_risk.head(15).index,
        y=country_risk.head(15)['mean'],
        error_y=country_risk.head(15)['std'],
        title=f"Average {main_risk_var.replace('_', ' ').title()} by Country",
        labels={'x': 'Country', 'y': f'Average {main_risk_var}'}
    )
    fig.update_xaxes(tickangle=45)
    fig.update_layout(height=500)
    fig.show()

print("\n✅ Bivariate analysis complete!")

In [ ]:
# Data Insights and Pattern Discovery
print("💡 DATA INSIGHTS & PATTERN DISCOVERY")
print("=" * 50)
print("🎯 SYNTHESIZING FINDINGS TO IDENTIFY NATURAL STORIES")

# Function to identify potential stories in the data
def discover_data_stories():
    """Analyze patterns to identify compelling data stories"""
    
    stories = []
    insights = []
    
    # 1. Geographic Inequality Story
    if 'country' in risk_data.columns and potential_risk_cols:
        main_risk = potential_risk_cols[0] if potential_risk_cols[0] in numeric_cols else numeric_cols[0]
        country_stats = risk_data.groupby('country')[main_risk].agg(['mean', 'count']).round(3)
        
        # Find countries with extreme values
        highest_risk = country_stats.loc[country_stats['mean'].idxmax()]
        lowest_risk = country_stats.loc[country_stats['mean'].idxmin()]
        
        risk_range = country_stats['mean'].max() - country_stats['mean'].min()
        risk_ratio = country_stats['mean'].max() / country_stats['mean'].min() if country_stats['mean'].min() > 0 else float('inf')
        
        stories.append({
            'title': 'Geographic Inequality Story',
            'description': f"Extreme variation in {main_risk} across countries",
            'key_facts': [
                f"Highest risk: {country_stats['mean'].idxmax()} ({highest_risk['mean']:.3f})",
                f"Lowest risk: {country_stats['mean'].idxmin()} ({lowest_risk['mean']:.3f})",
                f"Risk range: {risk_range:.3f}",
                f"Risk ratio: {risk_ratio:.1f}x difference"
            ],
            'story_potential': 'High' if risk_ratio > 2 else 'Medium'
        })
    
    # 2. Scale vs Severity Story
    if 'country' in risk_data.columns and len(potential_risk_cols) >= 2:
        # Look for population/economic exposure data
        exposure_cols = [col for col in numeric_cols if any(term in col.lower() 
                        for term in ['population', 'value', 'economic', 'gdp', 'vop'])]
        
        if exposure_cols:
            exposure_col = exposure_cols[0]
            risk_col = potential_risk_cols[0] if potential_risk_cols[0] in numeric_cols else numeric_cols[0]
            
            country_analysis = risk_data.groupby('country').agg({
                risk_col: 'mean',
                exposure_col: 'sum'
            }).round(3)
            
            # Find high-risk vs high-exposure countries
            high_risk_country = country_analysis[risk_col].idxmax()
            high_exposure_country = country_analysis[exposure_col].idxmax()
            
            stories.append({
                'title': 'Scale vs Severity Story',
                'description': f"Different types of vulnerability: severity vs exposure",
                'key_facts': [
                    f"Highest risk severity: {high_risk_country} ({country_analysis.loc[high_risk_country, risk_col]:.3f})",
                    f"Highest exposure: {high_exposure_country} ({country_analysis.loc[high_exposure_country, exposure_col]:,.0f})",
                    f"Same country?: {'Yes' if high_risk_country == high_exposure_country else 'No'}"
                ],
                'story_potential': 'High' if high_risk_country != high_exposure_country else 'Medium'
            })
    
    # 3. Distribution Pattern Story
    if potential_risk_cols:
        main_risk = potential_risk_cols[0] if potential_risk_cols[0] in numeric_cols else numeric_cols[0]
        risk_values = risk_data[main_risk].dropna()
        
        # Analyze distribution characteristics
        skewness = risk_values.skew()
        q25, q50, q75 = risk_values.quantile([0.25, 0.5, 0.75])
        extreme_high = (risk_values > q75 + 1.5 * (q75 - q25)).sum()
        extreme_low = (risk_values < q25 - 1.5 * (q75 - q25)).sum()
        
        stories.append({
            'title': 'Risk Distribution Pattern',
            'description': f"How {main_risk} is distributed across regions",
            'key_facts': [
                f"Distribution skew: {skewness:.2f} ({'Right-skewed' if skewness > 0.5 else 'Left-skewed' if skewness < -0.5 else 'Roughly normal'})",
                f"Median: {q50:.3f}",
                f"High outliers: {extreme_high:,} regions",
                f"Low outliers: {extreme_low:,} regions"
            ],
            'story_potential': 'High' if abs(skewness) > 1 or extreme_high > len(risk_values) * 0.05 else 'Medium'
        })
    
    # 4. Correlation Insights Story
    if len(numeric_cols) > 2:
        corr_matrix = risk_data[numeric_cols].corr()
        
        # Find strongest correlations (excluding self-correlations)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
        strong_corrs = corr_matrix.where(mask).stack()
        strongest_corr = strong_corrs.loc[strong_corrs.abs().idxmax()]
        strongest_pair = strong_corrs.abs().idxmax()
        
        # Count very strong correlations
        very_strong = (strong_corrs.abs() > 0.8).sum()
        strong = (strong_corrs.abs() > 0.6).sum()
        
        stories.append({
            'title': 'Variable Relationships Story',
            'description': "How different risk factors relate to each other",
            'key_facts': [
                f"Strongest correlation: {strongest_pair[0]} ↔ {strongest_pair[1]} ({strongest_corr:.3f})",
                f"Very strong correlations (>0.8): {very_strong}",
                f"Strong correlations (>0.6): {strong}",
                f"Relationship type: {'Positive' if strongest_corr > 0 else 'Negative'}"
            ],
            'story_potential': 'High' if abs(strongest_corr) > 0.7 else 'Medium'
        })
    
    return stories

# Discover and display stories
data_stories = discover_data_stories()

print(f"\n🎬 DISCOVERED {len(data_stories)} POTENTIAL DATA STORIES:")
print("=" * 60)

for i, story in enumerate(data_stories, 1):
    print(f"\n📖 STORY {i}: {story['title']}")
    print(f"Description: {story['description']}")
    print(f"Story Potential: {story['story_potential']}")
    print("Key Facts:")
    for fact in story['key_facts']:
        print(f"  • {fact}")

# Recommend the most compelling stories
high_potential_stories = [s for s in data_stories if s['story_potential'] == 'High']

print(f"\n⭐ RECOMMENDED STORIES FOR NARRATIVE DEVELOPMENT:")
print("=" * 60)

if high_potential_stories:
    for i, story in enumerate(high_potential_stories, 1):
        print(f"\n🌟 PRIORITY {i}: {story['title']}")
        print(f"Why compelling: {story['description']}")
        
        # Suggest narrative angle
        if 'inequality' in story['title'].lower():
            print("📝 Narrative angle: 'Tale of Two Extremes' - contrast countries with highest vs lowest risk")
        elif 'scale vs severity' in story['title'].lower():
            print("📝 Narrative angle: 'Different Faces of Vulnerability' - severity vs exposure comparison")
        elif 'distribution' in story['title'].lower():
            print("📝 Narrative angle: 'The Hidden Majority vs The Vulnerable Few' - outlier analysis")
        elif 'relationship' in story['title'].lower():
            print("📝 Narrative angle: 'Connected Crises' - how different factors compound risks")
else:
    print("All stories have medium potential - consider combining multiple stories for stronger narrative")

print(f"\n✅ Pattern discovery complete! {len(high_potential_stories)} high-potential stories identified.")
print("🎯 Ready to build authentic, data-driven narratives!")